## Computes number of atoms and minimum bond edit distance for all products 

In [1]:
from collections import defaultdict
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Atom, BondType

from tqdm.notebook import tqdm

import os
import SM_amats

In [2]:
if not os.path.exists("./data_files/smiles_with_all_dists"):
    os.makedirs("./data_files/smiles_with_all_dists")

### various utility functions

In [3]:
# change this for the different atoms
atoms = [6,6,6,6,6,7,8,8]

def molFromAdjMat(atoms, amat):
    """Creates a mol object from an adjacency matrix.
    Inputs:
    atoms: list of atomic numbers of atoms, by row
    amat: adjacency matrix. Has to have same length as atoms (obviously)
    Output: mol object
    """

    m = Chem.RWMol()
    # add in the separate atoms
    for a in atoms: m.AddAtom(Atom(a))
    side_len = len(amat)
    for r in range(side_len):
        for c in range(r+1,side_len):
            bond_order = amat[r][c]
            if bond_order > 0:
                if bond_order == 1: m.AddBond(r,c,BondType.SINGLE)
                if bond_order == 2: m.AddBond(r,c,BondType.DOUBLE)
                if bond_order == 3: m.AddBond(r,c,BondType.TRIPLE)
    try:
        Chem.SanitizeMol(m)
    except: 
        m = Chem.MolFromSmiles("C")
    return m

In [4]:
def canonize_smiles(s):
    return Chem.MolToSmiles(Chem.MolFromSmiles(s))


alphabet = "cnoCNO"
alphabet = [i for i in alphabet]
alphabet.sort()

# a quick way to get heavy atom count without going through the mol object
# only for CNO systems - if there are others, more atoms will need to be included in the alphabet variable.

def count_letters(s):
    # counts number of relevant letters in a string
    result = len([char for char in s if char in alphabet])
    return result

In [7]:
def compile_smiles_dists(file_index):
    
    """
    given a file index for product matrices, for all products, 
    compute graph edit distance from each starting material hybridization combination
    outputs a csv file.    
    """
    
    # load products
    atoms = [6,6,6,6,6,7,8,8]

    file_tag = str(file_index).zfill(2)
    amat_file = f"./product_amats/pdt_amat_{file_tag}_int8.npy"
    amats = np.load(amat_file)

    # make product smiles
    mols = [molFromAdjMat(atoms,amat) for amat in amats]
    smiles = [Chem.MolToSmiles(m) for m in mols]
    mols = []

    data_df_dict = {}
    data_df_dict["smiles"] = smiles
    
    hybrid_combos = ["ac2_am2","ac2_am3","ac3_am2","ac3_am3"]
    for hc in hybrid_combos:
        # load transformation file and take the sum of bond edits
        dmat_file = f"./rxn_mats/dmats_{hc}_{file_tag}.npy"
        dmats = np.load(dmat_file)
        bond_change_sums = [sum(sum(np.abs(dmat)))/2 for dmat in dmats]
        data_df_dict[hc] = bond_change_sums

    out_df = pd.DataFrame(data=data_df_dict)   
    out_df.to_csv(f"./data_files/smiles_with_all_dists/smiles_with_all_dists_{file_tag}.csv",index=False)

In [8]:
# 10 minutes on single core - can be pretty memory intensive
for i in tqdm(range(18)):
    compile_smiles_dists(i)

  0%|          | 0/18 [00:00<?, ?it/s]

### make a default dict that collects the minimum distance, for each time a certain SMILES appears

In [9]:
# less than 20 s
sd = defaultdict(list)

for file_index in tqdm(range(18)):
    
    file_tag = str(file_index).zfill(2)
    data = pd.read_csv(f"./data_files/smiles_with_all_dists/smiles_with_all_dists_{file_tag}.csv")
    dist_array = np.array(data[list(data)[1:]])
    
    # get minumum distances across all 4 hybridization combinations, for atoms =< 4 heavy atoms.
    data["min_dist_all"] = np.min(dist_array,axis=1)
    
    for r in data.itertuples():
        split_smiles = r.smiles.split(".")
    
        for ss in split_smiles:
                       
            atom_count = int(count_letters(ss))
            if atom_count >= 4:
                sd[ss].append(r.min_dist_all)
        
    data = []

  0%|          | 0/18 [00:00<?, ?it/s]

### canonize entries. Done here to minimize work duplication.

In [10]:
# less than 20 s
sd2 = defaultdict(list)
for k in tqdm(sd.keys()):
    canon_smiles = canonize_smiles(k)
    sd2[canon_smiles].append(np.min(sd[k]))
sd = []

  0%|          | 0/57230 [00:00<?, ?it/s]

In [11]:
data_raw = pd.DataFrame(data={"smiles":sd2.keys(),
                              "min_dist_all":[min(i) for i in sd2.values()],
                             "natoms":[count_letters(s) for s in sd2.keys()]})

data_raw.head()

,smiles,min_dist_all,natoms
0,COON,7.0,4
1,CONO,7.0,4
2,CON=O,8.0,4
3,NOCO,5.0,4
4,C1ONO1,6.0,4


In [12]:
data_raw.to_csv("./data_files/smiles_min_dist_natoms.csv")